In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("result.csv")

In [3]:
data.head()

,bikeNo,createDate,userGuid,faultType,desWordLen,hasOrderId,userRideCount,userRideTime,userIssueFaultCount,userConfirmedIssueFaultCount,...,recent1DayIssueFaultUserCount,recent1DayIssueFaultTypeCount,typefixResult,bikefixResult,maintUserGuid,maintUserName,updateDate,inputType,distance,fixResult
0,2100000064,20180331104334,a6cf99692e1945519b335510cd57ef28,111,0,1,3,921000,0,0,...,1,1,1,1,d673f48c7eb74a13b55443333d0e6b51,上海奉贤运维刘宇,2.018033e+13,2.0,3045.919635,1.0
1,2100000070,20180331161752,3a888b7ff5994ec29c2c355416895c3a,112,0,1,1,3866000,0,0,...,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2100000094,20180316115546,7c06f81ca312407f8b1e5b1f9a3342f6,106,0,1,0,0,0,0,...,1,1,1,1,69a1c504e88d41238c04022f96034888,闵行定点刘哲,2.018033e+13,1.0,22.697366,1.0
3,2100000110,20180315154604,987be31d982c4850a9b503d054c73a62,109,0,0,10,3149000,0,0,...,1,1,1,1,d673f48c7eb74a13b55443333d0e6b51,上海奉贤运维刘宇,2.018032e+13,1.0,37.852437,1.0
4,2100000208,20180331152134,bb7f976e6a4c4226ae923c13800aca11,104,0,1,1,1241000,0,0,...,1,1,1,1,d673f48c7eb74a13b55443333d0e6b51,上海奉贤运维刘宇,2.018040e+13,1.0,7.831931,1.0


In [3]:
# df = data.filter(regex="faultType|desWordLen|hasOrderId|user.*|all.*|rencent7Day.*|afterFault3Hour.*|idleHour|serveDay|recent1DayIssueFaultUserCount|recent1DayIssueFaultTypeCount|bikefixResult")

In [12]:
df = data.drop(["bikeNo", "createDate", "typefixResult", "userGuid", "userHasMonthCard", "maintUserGuid", "maintUserName", "fixResult", "updateDate"], axis=1)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839605 entries, 0 to 839604
Data columns (total 61 columns):
faultType                          839605 non-null int64
desWordLen                         839605 non-null int64
hasOrderId                         839605 non-null int64
userRideCount                      839605 non-null int64
userRideTime                       839605 non-null int64
userIssueFaultCount                839605 non-null int64
userConfirmedIssueFaultCount       839605 non-null int64
allAverageRideTime                 839605 non-null int64
allAverageRideDistance             839605 non-null int64
allRideCount                       839605 non-null int64
allAverateRideSpeed                839605 non-null float64
recent1DayRideCount                839605 non-null int64
recent1DayAverageRideTime          839605 non-null int64
recent1DayAverageDistance          839605 non-null int64
recent1DayRideSpeed                839605 non-null float64
recent3DayRideCount           

In [15]:
y = df.pop("bikefixResult")

In [18]:
X = df.as_matrix()
y = y.as_matrix()

In [19]:
X.shape

(839605, 60)

In [20]:
y.shape

(839605,)

In [21]:
X_train = X[:600000, :]
X_test = X[600000:, :]
y_train = y[:600000]
y_test = y[600000:]

In [24]:
y_test.shape

(239605,)

In [ ]:
from xgboost import XGBClassifier
parms = [5, 6, 8]
for parm in parms:    
    clf = XGBClassifier(max_depth=7, learning_rate=0.14)
    clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    d1 = pd.DataFrame(y_test, columns=["test"])
    d2 = pd.DataFrame(y_pred, columns=["pred"])
    d = pd.concat([d1, d2], axis=1)
    count1 = d[d.pred == d.test].shape[0]
    count = d.shape[0]
    print(count1/count)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7787441831347426


In [64]:
d1 = pd.DataFrame(y_test, columns=["train"])
d2 = pd.DataFrame(y_pred, columns=["pred"])
d = pd.concat([d1, d2], axis=1)

In [65]:
d["t"] = d.train - d.pred

In [30]:
count = 0

for i in range(0, 239605):
    df = d.iloc[i:i+1]
    if list(df.t)[0] == -1:
        count += 1
print(count/239605)

0.13239707017800129


In [69]:
d[(d.pred == 0) & (d.train == 1)]

,train,pred,t
6,1,0,1
11,1,0,1
22,1,0,1
24,1,0,1
45,1,0,1
46,1,0,1
47,1,0,1
65,1,0,1
97,1,0,1
106,1,0,1


In [35]:
132877/(132877+31723)

0.8072721749696233

In [43]:
(326683+230978)/(326683+230978+24383+17956)

0.929435

In [52]:
(294586+194073)/(294586+194073+61288+50053)

0.8144316666666667

In [61]:
(133491+53180)/(133491+53180+31281+21653)

0.779078065983598

In [70]:
(133617+52991)/(133617+52991+31470+21527)

0.7788151332401244

In [75]:
for i in range(3, 10, 2):
    print(i)

3
5
7
9
